In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Input, Dropout
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

from keras.layers import concatenate

Using TensorFlow backend.


## Parameters

In [3]:
PREFILTER = True
PREFILTER_THRESHOLD = 0.2 # Data excluded if Brand variance greather than 0.2M or less than -0.2M

UNDERSAMPLE = True
UNDERSAMPLE_RATIO = 0.6 #  Amount of data with no comment =  Amount of data with comment / UNDERSAMPLE_RATIO
EXTRA_TEST_DATA = 100 # Amount of data thrown out when undersampling to include in the test set

AUGMENT_DATA = True

DATA = ["Variance"] # "Variance", "All", 
MODEL = "LSTM"    # 


## Loading Data

commentaries_to_pickle.py to save commentary spreedsheet to pickle

In [4]:
from helper_save_load import load_from_pickle

df_c = load_from_pickle("data/Labeled_comments2.pkl")

In [5]:
# These brands are in df_c but do not appear in the variance dataframe df_v
ignored_brands = ["IGNORE",
"57-CAMAY BAR SOAP Brand",
"71-LBP SH/CD Roll-up Brand",
"72-LBP SHOWER Roll-up Brand"]

df_c = df_c[~df_c["Brand_1"].isin(ignored_brands)]

In [6]:
df_c.insert(16, "WithComm", ~df_c["Commentaries"].isnull())
df_c.insert(17, "Other", (df_c[['Promo', 'Phasing', 'SP&D', 'POS']].sum(axis=1) == 0) & df_c['WithComm'])
df_c["Month"] = pd.to_datetime(df_c["Month"]).dt.strftime("%b_%Y")

In [7]:
label_cols = ['Promo', 'Phasing', 'SP&D', 'POS', 'Other', 'WithComm']

territory_label_cols = df_c.columns[18:31].to_list()
territory_labels = df_c[territory_label_cols]

In [8]:
df_a, df_f, df_v = load_from_pickle("data/dataframes_Dollars.pickle")
del df_a, df_f

In [9]:
territories = [territory for territory, values in df_v.groupby(['Territory']).groups.items()]

In [ ]:
df_pos = load_from_pickle("data/pos_with_brands.pickle")

## Pre-Filtering Data
Including data if variance is greater than the 0.2M threshold

In [10]:
test = df_c[(~df_c["Num"].duplicated())]
len(test[test["WithComm"] == 0]) / len(test) # Fraction of number of brand-month pairs with no commentaries

0.7415946205571565

In [11]:
if PREFILTER:
    df_c = df_c[(df_c["Variance"] >= PREFILTER_THRESHOLD) | (df_c["Variance"] <= -PREFILTER_THRESHOLD)]

In [12]:
test = df_c[(~df_c["Num"].duplicated())]
len(test[test["WithComm"] == 0]) / len(test) #Fraction of number of brand-month pairs with no commentaries after filtering

0.12237762237762238

## Removing multiclass labels

Rows with multiple positive class labels are changed to a single label according to the importance of the class

In [13]:
CLASS_PRIORITY = ["Promo", "SP&D", "Phasing", "POS"] # Most important class to predict to least

print('Number of multiclass labels before:', sum(df_c[label_cols[:4]].sum(axis=1) > 1))
      
for idx, row in df_c[df_c[label_cols[:4]].sum(axis=1) > 1][label_cols[:4]].iterrows():
    positive_label_idx = min([CLASS_PRIORITY.index(c) for c in row[row > 0].index])
    negative_labels = CLASS_PRIORITY[:]
    negative_labels.pop(positive_label_idx) # List of columns to set to zero
    df_c.loc[idx, negative_labels] = 0

print('Number of multiclass labels after:', sum(df_c[label_cols[:4]].sum(axis=1) > 1))

Number of multiclass labels before: 38
Number of multiclass labels after: 0
